Essentials

In [ ]:
import io
import numpy as np
import tensorflow 
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, GRU
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback

**Fetching the dataset** 

Lexicons for Latin-Tamil are taken from Google's Dakshina dataset. The necessary datasets have been uploaded to github, cloned and used for the reminder of the code.

In [ ]:
!git clone https://github.com/borate267/lexicon-dataset.git

Cloning into 'lexicon-dataset'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


In [49]:
# GLOBAL VARIABLES

print_data = False

Reading the dataset


In [35]:
train_dir = "lexicon-dataset/ta.translit.sampled.train.tsv"
dev_dir = "lexicon-dataset/ta.translit.sampled.dev.tsv"
test_dir = "lexicon-dataset/ta.translit.sampled.test.tsv"

# The following function reads the raw text document and returns a list of lists comprising the romanized and native versions of the words

def read_corpus(corpus_file):
  tamil_words = []
  latin_words = []
  with io.open(corpus_file, encoding ='utf-8') as f:
    for line in f:
      if '\t' not in line:
        continue
      tokens = line.rstrip().split("\t")
      latin_words.append(tokens[1])
      tamil_words.append(tokens[0])
  return latin_words, tamil_words

train_source, train_target = read_corpus(train_dir)
valid_source, valid_target = read_corpus(dev_dir)
test_source, test_target = read_corpus(test_dir)

print("Number of training samples: ", len(train_source))
print("Number of validation samples: ", len(valid_source))
print("Number of testing samples: ", len(test_source))




Number of training samples:  68218
Number of validation samples:  6827
Number of testing samples:  6864


In [50]:
# Pre-processing data

# The following lists contain the source and target words that are to
# be used for training and validation

train_source = []
train_target = []
val_source = []
val_target = []

# The following entities hold the vocabulary for Tamil and latin languages
vocab_source = set()
vocab_target = set()

# Procuring training data
with io.open(train_dir, encoding ='utf-8') as f:
  for line in f:
    if '\t' not in line:
      continue
    tokens = line.rstrip().split("\t")
    input_text = tokens[1]
    target_text = tokens[0]
    train_source.append(input_text)
    train_target.append(target_text)

    # Creating vocabulary
    for char in input_text:
        if char not in vocab_source:
            vocab_source.add(char)
    for char in target_text:
        if char not in vocab_target:
            vocab_target.add(char)

# Procuring Validation data
with io.open(dev_dir, encoding ='utf-8') as f:
  for line in f:
    if '\t' not in line:
      continue
    tokens = line.rstrip().split("\t")
    input_text = tokens[1]
    target_text = tokens[0]
    val_source.append(input_text)
    val_target.append(target_text)

    # Updating vocabulary
    for char in input_text:
        if char not in vocab_source:
            vocab_source.add(char)
    for char in target_text:
        if char not in vocab_target:
            vocab_target.add(char)

vocab_source = sorted(list(vocab_source))
vocab_target = sorted(list(vocab_target))
num_encoder_tokens = len(vocab_source)
num_decoder_tokens = len(vocab_target)
max_encoder_seq_length = max([len(txt) for txt in train_source])
max_decoder_seq_length = max([len(txt) for txt in train_target])

if (print_data):
    print("Number of samples:", len(train_source))
    print("Number of unique input tokens:", num_encoder_tokens)
    print("Number of unique output tokens:", num_decoder_tokens)
    print("Max sequence length for inputs:", max_encoder_seq_length)
    print("Max sequence length for outputs:", max_decoder_seq_length)

# Creating tokens for vocabulary
input_token_index = dict([(char, i) for i, char in enumerate(vocab_source)])
target_token_index = dict([(char, i) for i, char in enumerate(vocab_target)])


Character Embedding

**Encoded Source Sequences**: Padded to a maximum length of max_encSeqLen characters with a vocabulary of sizeofLatinVocab different characters. 
**SHAPE: (len(train_source), max_encSeqLen, sizeofLatinVocab)**

**Encoded Target Sequences**: Padded to a maximum length of max_decSeqLen characters with a vocabulary of sizeofTamilVocab different characters. 
**SHAPE: (len(train_source), max_decSeqLen, sizeofTamilVocab)**

In [59]:
def generate_batch(X, Y):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_encoder_seq_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_decoder_seq_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_decoder_seq_length, num_decoder_tokens), dtype="float32")
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], Y[j:j+batch_size])):
                for t, char in enumerate(input_text):
                    encoder_input_data[i, t] = input_token_index[char]
                for t, char in enumerate(target_text):
                    decoder_input_data[i, t] = target_token_index[char]
                    if t > 0:
                        decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

             

Configuring the Sweep Hyperparameter dictionary

In [56]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 128  # Latent dimensionality of the encoding space.

Defining the model

In [72]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, ))
enc_emb =  Embedding(num_encoder_tokens, latent_dim , mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that takes encoder and decoder input 
# to output decoder_outputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()



Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 128)    3328        input_27[0][0]                   
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 128)    5888        input_28[0][0]                   
___________________________________________________________________________________________

In [73]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
history=model.fit_generator(generator = generate_batch(train_source, train_target),
                    steps_per_epoch = len(train_source)//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(val_source, val_target),
                    )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1065/1065 [==============================] - ETA: 0s - loss: 0.7693 - acc: 0.2665